In [2]:
#training.py(trainig the chatbot)

import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

lemmatizer = WordNetLemmatizer
intents = json.load(open("intents.json"))

words = []
classes = []
documents = []
ignore_letters = ['?','!','.',',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)  # découper la phrase en mots
        words.append(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

print(documents)


[(['hello'], 'greetings'), (['hey'], 'greetings'), (['hi'], 'greetings'), (['good', 'day'], 'greetings'), (['greetings'], 'greetings'), (['what', "'s", 'up', '?'], 'greetings'), (['how', 'is', 'it', 'going', '?'], 'greetings'), (['goodbye'], 'goodbye'), (['bye'], 'goodbye'), (['see', 'you', 'later'], 'goodbye'), (['farewell'], 'goodbye'), (['take', 'care'], 'goodbye'), (['how', 'old', 'are', 'you', '?'], 'age'), (['what', "'s", 'your', 'age', '?'], 'age'), (['when', 'were', 'you', 'born', '?'], 'age'), (['what', "'s", 'your', 'name', '?'], 'name'), (['who', 'are', 'you', '?'], 'name'), (['do', 'you', 'have', 'a', 'name', '?'], 'name'), (['where', 'can', 'I', 'shop', '?'], 'shop'), (['recommend', 'a', 'shop'], 'shop'), (['best', 'places', 'to', 'shop'], 'shop'), (['what', 'are', 'your', 'hours', '?'], 'hours'), (['when', 'are', 'you', 'open', '?'], 'hours'), (['hours', 'of', 'operation'], 'hours'), (['stock', 'availability'], 'stocks'), (['check', 'stocks'], 'stocks'), (['inventory', 's

In [3]:

import random
import json
import pickle
import numpy as np
nltk.download('wordnet')


import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

lemmatizer = WordNetLemmatizer()
intents = json.load(open("intents.json"))

words = []
classes = []
documents = []
ignore_letters = ['?','!','.',',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)  # découper la phrase en mots
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))
print(words)


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\FUJITSU\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


["'s", 'I', 'a', 'age', 'are', 'availability', 'best', 'born', 'bye', 'can', 'care', 'check', 'day', 'do', 'farewell', 'going', 'good', 'goodbye', 'greeting', 'have', 'hello', 'hey', 'hi', 'hour', 'how', 'inventory', 'is', 'it', 'later', 'name', 'of', 'old', 'open', 'operation', 'place', 'recommend', 'see', 'shop', 'status', 'stock', 'take', 'to', 'up', 'were', 'what', 'when', 'where', 'who', 'you', 'your']


In [15]:
import random
import json
import pickle
import numpy as np


import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

lemmatizer = WordNetLemmatizer()
intents = json.load(open("intents.json"))

words = []
classes = []
documents = []
ignore_letters = ['?','!','.',',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)  # découper la phrase en mots
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

classes = sorted(set(classes))
#save wors, classes in pickle file
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))
# Machine learning part
training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]

    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1

    training.append([bag, output_row])

# Shuffle the training data
random.shuffle(training)

# Separate features (X) and labels (Y)
X = np.array([bag for bag, _ in training])
Y = np.array([output_row for _, output_row in training])

# model (neural network)
model = Sequential()
model.add(Dense(128, input_shape=(X.shape[1],), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(Y.shape[1], activation="softmax"))

# Use the updated parameters for the SGD optimizer
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=['accuracy'])

model.fit(X, Y, epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.model')
print('Done')

Epoch 1/200
6/6 [==============================] - 1s 22ms/step - loss: 1.9764 - accuracy: 0.2222
Epoch 2/200
6/6 [==============================] - 0s 8ms/step - loss: 1.9021 - accuracy: 0.2963
Epoch 3/200
6/6 [==============================] - 0s 6ms/step - loss: 1.8870 - accuracy: 0.3704
Epoch 4/200
6/6 [==============================] - 0s 7ms/step - loss: 1.8022 - accuracy: 0.3704
Epoch 5/200
6/6 [==============================] - 0s 5ms/step - loss: 1.8364 - accuracy: 0.2963
Epoch 6/200
6/6 [==============================] - 0s 4ms/step - loss: 1.7794 - accuracy: 0.3333
Epoch 7/200
6/6 [==============================] - 0s 4ms/step - loss: 1.6687 - accuracy: 0.3704
Epoch 8/200
6/6 [==============================] - 0s 4ms/step - loss: 1.7432 - accuracy: 0.3333
Epoch 9/200
6/6 [==============================] - 0s 4ms/step - loss: 1.6751 - accuracy: 0.4074
Epoch 10/200
6/6 [==============================] - 0s 4ms/step - loss: 1.6285 - accuracy: 0.4074
Epoch 11/200
6/6 [==========

INFO:tensorflow:Assets written to: chatbot_model.model\assets


Done


In [ ]:
#chatbot.py
import random
import json
import pickle
import numpy as np
from tensorflow.keras.models import load_model

import nltk
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()#réduire les mots à leur forme de base ou racine.
intents = json.load(open("intents.json"))

words = pickle.load(open("words.pkl", "rb"))
classes = pickle.load(open("classes.pkl", "rb"))
model = load_model("chatbot_model.model")

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)  # tokenizes the input sentence into a list of words
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]  # Lemmatization involves reducing words to their base or root form
    return sentence_words

def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)  
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25 
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result
print("Go! Bot is running!")
while True:
    message = input("")
    ints = predict_class(message)
    res = get_response(ints, intents)
    print(res)

Go! Bot is running!


 hi


1/1 [==============================] - 0s 80ms/step
hello!


 how are you


1/1 [==============================] - 0s 32ms/step
I don't have an age. I'm just a computer program.


...

1/1 [==============================] - 0s 31ms/step
hello!


 what's your age?


1/1 [==============================] - 0s 25ms/step
I don't have an age. I'm just a computer program.


 do you have a name?


1/1 [==============================] - 0s 31ms/step
I'm a chatbot, you can call me ChatGPT.
